## Import Libraries

In [48]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor


In [49]:
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

y = np.log1p(train["SalePrice"])   # log(1 + SalePrice)
train.drop("SalePrice", axis=1, inplace=True)

In [50]:
all_data = pd.concat([train, test], axis=0)


In [51]:
# Age of house
all_data["HouseAge"] = all_data["YrSold"] - all_data["YearBuilt"]
all_data["RemodAge"] = all_data["YrSold"] - all_data["YearRemodAdd"]

# Save test_ids before dropping 'Id' from all_data
# Use original 'test' DataFrame for robust extraction of test_ids
test_ids = test["Id"]

# Drop Id from all_data
# Use errors='ignore' for robustness against multiple cell executions
all_data.drop("Id", axis=1, inplace=True, errors='ignore')

# Split all_data back into train and test after feature engineering
# Use len(y) to get the original length of the training data reliably
original_train_len = len(y)
train = all_data.iloc[:original_train_len, :] # Redefine train with engineered features
test = all_data.iloc[original_train_len:, :] # Redefine test with engineered features

In [52]:
num_features = train.select_dtypes(include=["int64", "float64"]).columns
cat_features = train.select_dtypes(include=["object"]).columns

In [53]:
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot",
     __import__("sklearn").preprocessing.OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", num_pipeline, num_features),
    ("cat", cat_pipeline, cat_features)
])


In [54]:
model = GradientBoostingRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=4,
    random_state=42
)


In [55]:
pipeline = Pipeline([
    ("prep", preprocessor),
    ("model", model)
])


In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmse = np.sqrt(-cross_val_score(
    pipeline, train, y,
    scoring="neg_mean_squared_error",
    cv=kf
))

print("CV RMSE:", rmse.mean())


In [ ]:
pipeline.fit(train, y)

preds = pipeline.predict(test)
preds = np.expm1(preds)   # reverse log


In [ ]:
if 'preds' not in locals() and 'preds' not in globals():
    raise NameError("The 'preds' variable is not defined. Please ensure the model fitting and prediction cell (__Gc139rZNQS) has been executed before running this cell.")

submission = pd.DataFrame({
    "Id": test_ids,
    "SalePrice": preds
})

submission.to_csv("submission.csv", index=False)
submission.head()

# Task
Generate advanced features (e.g., polynomial, interaction terms, or domain-specific aggregations) from the existing features in the `all_data` DataFrame. Ensure to update the `num_features` and `cat_features` lists to reflect these new features, and then re-evaluate the model performance.

## Generate Advanced Features

### Subtask:
Create more complex features by combining existing ones, such as total living area, total bathrooms, and interaction terms. After generating these new features, ensure to update the `num_features` and `cat_features` lists to reflect them.


**Reasoning**:
I will create new advanced features like `TotalSF`, `TotalBath`, `OverallQual_GrLivArea`, and `HasPool` by combining existing columns in the `all_data` DataFrame. After creating these new features, I will re-split the `all_data` DataFrame into `train` and `test` based on `original_train_len` to ensure both DataFrames include the newly engineered features. Finally, I will update the `num_features` and `cat_features` lists based on the data types in the re-split `train` DataFrame.



In [ ]:
all_data["TotalSF"] = all_data["GrLivArea"] + all_data["TotalBsmtSF"]
all_data["TotalBath"] = (
    all_data["FullBath"]
    + 0.5 * all_data["HalfBath"]
    + all_data["BsmtFullBath"]
    + 0.5 * all_data["BsmtHalfBath"]
)
all_data["OverallQual_GrLivArea"] = all_data["OverallQual"] * all_data["GrLivArea"]
all_data["HasPool"] = (all_data["PoolArea"] > 0).astype(int)

# Re-split all_data back into train and test after feature engineering
train = all_data.iloc[:original_train_len, :]
test = all_data.iloc[original_train_len:, :]

# Update feature lists
num_features = train.select_dtypes(include=np.number).columns
cat_features = train.select_dtypes(include=object).columns

print("Advanced features created and train/test DataFrames updated.")
print(f"New num_features count: {len(num_features)}")
print(f"New cat_features count: {len(cat_features)}")

**Reasoning**:
With the `num_features` and `cat_features` lists now updated to include the newly engineered features, I need to redefine the `preprocessor` to incorporate these changes. This ensures that the data transformation steps (imputation, scaling, one-hot encoding) are applied correctly to all relevant features, including the new ones, before the model is trained.



In [ ]:
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot",
     __import__("sklearn").preprocessing.OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", num_pipeline, num_features),
    ("cat", cat_pipeline, cat_features)
])


**Reasoning**:
With the `preprocessor` updated to handle the new features, the model pipeline now needs to be re-assembled to incorporate this updated `preprocessor`. This ensures that the entire data processing and modeling workflow uses the most current feature set before model evaluation.



In [ ]:
pipeline = Pipeline([
    ("prep", preprocessor),
    ("model", model)
])

print("Model pipeline re-assembled with updated preprocessor.")

**Reasoning**:
Now that the pipeline has been re-assembled with the updated preprocessor and new features, I need to re-evaluate the model's performance using cross-validation. This will allow us to assess if the advanced features have improved the model's predictive power.



In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmse = np.sqrt(-cross_val_score(
    pipeline, train, y,
    scoring="neg_mean_squared_error",
    cv=kf
))

print("CV RMSE (after advanced features):", rmse.mean())

## Implement Hyperparameter Tuning

### Subtask:
Utilize techniques like GridSearchCV or RandomizedSearchCV from scikit-learn to systematically search for the optimal hyperparameters for the `GradientBoostingRegressor`. This involves defining a grid of parameter values and using cross-validation to find the combination that yields the best performance.


**Reasoning**:
To implement hyperparameter tuning, I will import `GridSearchCV`, define a parameter grid for the `GradientBoostingRegressor`, instantiate `GridSearchCV` with the defined pipeline and parameters, fit it to the training data, and then print the best parameters and corresponding RMSE score.



In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {
    'model__n_estimators': [100, 300, 500],
    'model__learning_rate': [0.01, 0.05, 0.1],
    'model__max_depth': [3, 4, 5]
}

# Instantiate GridSearchCV
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=kf,
    scoring="neg_mean_squared_error",
    n_jobs=-1, # Use all available cores
    verbose=1
)

print("Starting GridSearchCV...")
# Fit GridSearchCV to the training data
grid_search.fit(train, y)

print("GridSearchCV completed.")
# Print the best parameters
print("Best hyperparameters found:", grid_search.best_params_)

# Print the best cross-validation score (RMSE)
best_rmse = np.sqrt(-grid_search.best_score_)
print("Best CV RMSE:", best_rmse)

## Re-evaluate Model Performance

### Subtask:
Re-run the cross-validation step to measure the improved RMSE using the optimal hyperparameters found during GridSearchCV.


**Reasoning**:
After running `GridSearchCV` to find the optimal hyperparameters, I will extract the best estimator from the `grid_search` object. This best estimator is a pipeline that includes the preprocessor and the `GradientBoostingRegressor` with the tuned parameters. I will then use this `best_estimator_` to perform a 5-fold cross-validation on the training data (`train` and `y`) to assess its performance with the optimized settings. Finally, I will calculate and print the mean of the Root Mean Squared Error (RMSE) obtained from this cross-validation, allowing for a direct comparison with previous evaluations.



In [ ]:
best_pipeline = grid_search.best_estimator_

kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmse_tuned = np.sqrt(-cross_val_score(
    best_pipeline, train, y,
    scoring="neg_mean_squared_error",
    cv=kf
))

print("CV RMSE (after hyperparameter tuning):", rmse_tuned.mean())

## Generate New Submission File

### Subtask:
Train the final model with the optimal hyperparameters on the entire training dataset (with new features) and generate a new prediction file for submission.


**Reasoning**:
Now that the optimal hyperparameters have been found and the model performance re-evaluated, I will train the `best_pipeline` on the entire training data, generate predictions on the test set, reverse the log transformation, and create the submission file.



In [ ]:
best_pipeline.fit(train, y)

preds_tuned = best_pipeline.predict(test)
preds_tuned = np.expm1(preds_tuned)   # reverse log

submission_tuned = pd.DataFrame({
    "Id": test_ids,
    "SalePrice": preds_tuned
})

submission_tuned.to_csv("submission_tuned.csv", index=False)

print("New submission file 'submission_tuned.csv' created.")
submission_tuned.head()

## Final Task

### Subtask:
Summarize the improvements made to the model, including the impact of new features and hyperparameter tuning on the model's performance and ranking potential.


## Summary:

### Data Analysis Key Findings

*   **Advanced Feature Engineering:** Four new features (`TotalSF`, `TotalBath`, `OverallQual_GrLivArea`, and `HasPool`) were created. This updated the feature set to 42 numerical and 43 categorical features.
*   **Performance Post-Feature Engineering:** After incorporating advanced features, the model achieved an average cross-validation RMSE of approximately 0.1265.
*   **Hyperparameter Tuning Impact:** Hyperparameter tuning using `GridSearchCV` on the `GradientBoostingRegressor` identified optimal settings, further reducing the average cross-validation RMSE to approximately 0.1254. This represents an improvement of about 0.0011 over the model with only advanced features.
*   **Final Model & Submission:** A final model, incorporating both advanced features and optimal hyperparameters, was trained on the entire dataset, and a submission file named 'submission\_tuned.csv' was successfully generated.

### Insights or Next Steps

*   The combination of targeted feature engineering and systematic hyperparameter tuning significantly enhanced the model's predictive performance, as evidenced by the consistent reduction in RMSE.
*   Future efforts could explore additional complex feature interactions, alternative regression models, or ensemble techniques to potentially achieve even greater accuracy.
